In [ ]:
!pip install neowise

In [ ]:
import neowise as nw
from sklearn.datasets import make_circles
import numpy as np
import math
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from matplotlib import cm

In [ ]:
# Data Preparation --- Make Circles
N_SAMPLES = 1000
TEST_SIZE = 0.1
X_data, y_data_t = make_circles(n_samples = N_SAMPLES, noise=0.05, random_state=100)
y_data = y_data_t.reshape(N_SAMPLES,1)
m = int(X_data.shape[0])
m_tr = int(math.ceil((90/100)*m))
m_cv = int(math.ceil((5/100)*m))
m_te = m - (m_tr + m_cv)
X_tr = np.zeros((m_tr,X_data.shape[1]))
y_tr_t = np.zeros((m_tr,1))
X_cv = np.zeros((m_cv,X_data.shape[1]))
y_cv_t = np.zeros((m_cv,1))
X_te = np.zeros((m_te,X_data.shape[1]))
y_te_t = np.zeros((m_te,1))
perm = np.random.permutation(m)
p = perm.reshape(m,1)
for i in range(0,m_tr):
  X_tr[i] = X_data[p[i]]
  y_tr_t[i] = y_data[p[i]]
for i in range(0,m_cv):
  X_cv[i] = X_data[p[i+m_tr]]
  y_cv_t[i] = y_data[p[i+m_tr]]
for i in range(0,m_te):
  X_te[i] = X_data[p[i+m_tr+m_cv]]
  y_te_t[i] = y_data[p[i+m_tr+m_cv]]
y_tr = y_tr_t.T
y_cv = y_cv_t.T
y_te = y_te_t.T
 
# Plotting the data
df = DataFrame(dict(x=X_data[:,0], y=X_data[:,1], label=y_data_t))
colors = {0:'red', 1:'blue'}
fig, ax = plt.subplots()
grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
plt.show()

In [ ]:
model = nw.Model(X_tr,y_tr,X_te,y_te,X_cv,y_cv)

In [ ]:
model.reset()
model.add("dense1",X_tr.shape[1],500,"relu")
model.add("dense2",500,250,"relu")
model.add("dense3",250,150,"tanh")
model.add("dense4",150,100,"tanh")
model.add("dense5",100,60,"tanh")
model.add("dense6",60,30,"relu")
model.add("dense7",30,15,"tanh")
model.add("dense8",15,1,"sigmoid")

In [ ]:
model.summary()

In [ ]:
model.fit(X_tr,y_tr,0.005,50,"Momentum","Binary",mb=16,alpha_decay=True)

In [ ]:
model.test(X_te,y_te,"Binary")

In [ ]:
model.plot("Cost")

In [ ]:
model.plot("Accuracy")

## Replace contents of argument "direc" with the directory you want to save the images!

## For saving and loading the model, pass the directory in which you want to save the file as the argument and append it by the name of the file with .h5 extension!

In [ ]:
model.plot("Cost",animate=True,direc='/content/drive/My Drive/neowise/circles/circles-costs/')

In [ ]:
model.plot("Accuracy",animate=True,direc='/content/drive/My Drive/neowise/circles/circles-accu/')

In [ ]:
model.save_model('/content/drive/My Drive/neowise/circles/circles.h5')

In [ ]:
model1 = nw.Model(X_tr,y_tr,X_te,y_te,X_cv,y_cv)

In [ ]:
model1.load_model('/content/drive/My Drive/neowise/circles/circles.h5')

In [ ]:
model1.summary()

In [ ]:
model1.test(X_te,y_te,"Binary")

In [ ]:
sns.set_style("whitegrid")
GRID_X_START = 2.5
GRID_X_END = -1.25
GRID_Y_START = 1.5
GRID_Y_END = -1.5
OUTPUT_DIR = "/content/drive/My Drive/neowise/circles/circles-vis"
grid = np.mgrid[GRID_X_START:GRID_X_END:100j,GRID_X_START:GRID_Y_END:100j]
grid_2d = grid.reshape(2,-1)
XX, YY = grid
def make_plot(X, y, plot_name, file_name=None, XX=None, YY=None, preds=None, dark=False):
    if (dark):
        plt.style.use('dark_background')
    else:
        sns.set_style("whitegrid")
    plt.figure(figsize=(16,12))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if(XX is not None and YY is not None and preds is not None):
        plt.contourf(XX, YY, preds.reshape(XX.shape), 25, alpha = 1, cmap=cm.Spectral)
        plt.contour(XX, YY, preds.reshape(XX.shape), levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=40, cmap=plt.cm.Spectral, edgecolors='black')
    if(file_name):
        plt.savefig(file_name)
        plt.close()
import os
def callback_numpy_plot(index, init_params):
    plot_title = "Iteration {:05}".format(index)
    file_name = "numpy_model_{:05}.png".format(index//1)
    file_path = os.path.join(OUTPUT_DIR, file_name)
    act = model.forward_prop(np.transpose(grid_2d),train_model=False)
    prediction_probs = act["A" + str(len(model.layer_names))]
    prediction_probs = prediction_probs.reshape(prediction_probs.shape[1], 1)
    make_plot(X_te, y_te, plot_title, file_name=file_path, XX=XX, YY=YY, preds=prediction_probs, dark=True)

In [ ]:
model.reset()
model.add("dense1",X_tr.shape[1],500,"relu")
model.add("dense2",500,250,"relu")
model.add("dense3",250,150,"tanh")
model.add("dense4",150,100,"tanh")
model.add("dense5",100,60,"tanh")
model.add("dense6",60,30,"relu")
model.add("dense7",30,15,"tanh")
model.add("dense8",15,1,"sigmoid")

model.fit(X_tr,y_tr,0.005,50,"Momentum","Binary",mb=16,alpha_decay=True,print_cost=False,callback=callback_numpy_plot)

In [ ]:
act = model.forward_prop(np.transpose(grid_2d),train_model=True)
prediction_probs_np = act["A" + str(len(model.layer_names))]
prediction_probs_np = prediction_probs_np.reshape(prediction_probs_np.shape[1], 1)
make_plot(X_te, y_te, "Final Iteration", file_name=None, XX=XX, YY=YY, preds=prediction_probs_np, dark=True)